<a href="https://colab.research.google.com/github/Vicky-0222/NLP/blob/master/lab3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Нейронные сети для преобразования текста**

Импорт необходимых библиотек

In [1]:
import numpy as np
import pandas as pd

### Создать нейронную сеть с нуля, т.е. не используя готовые библиотеки. Пример работы на любом табличном датасете.


Функция для линейной регрессии

In [2]:
def linear_regression(X: np.ndarray, weights: np.ndarray, bi) -> np.ndarray:
    return np.dot(X, weights) + bias

Функция для активации (сигмоид)

In [3]:
def activation_func(x: np.ndarray) -> np.ndarray:
    return 1 / (1 + np.exp(-x))

Функция для нейрона

In [4]:
def neuron(X: np.ndarray, weights: np.ndarray, bias: float) -> np.ndarray:
    temp_result = linear_regression(X, weights, bias)
    result = activation_func(temp_result)
    return result

Пример работы

In [5]:
data = {
        'feature1': [0.1, 0.2, 0.3, 0.4, 0.5],
        'feature2': [0.5, 0.4, 0.3, 0.2, 0.1],
        'label': [0, 0, 1, 1, 1]
    }

df1 = pd.DataFrame(data)

In [6]:
X = df1[['feature1', 'feature2']].values # Входные признаки
y = df1['label'].values # Целевая переменная

In [7]:
weights = np.random.rand(X.shape[1]) # Генерация весов
bias = np.random.rand() # Генерация смещений

In [8]:
output = neuron(X, weights, bias)
print(output)

[0.72644526 0.73337849 0.74019883 0.74690493 0.75349556]


### Сделать класс, в котором реализована возможность задать количество нейронов в скрытом слое и провести обучение.


In [9]:
class NeuralNetwork:
    def __init__(self, input_size, hidden1_size, hidden2_size, output_size, learning_rate=0.01):
        np.random.seed(42)
        self.lr = learning_rate

        # Инициализация весов и смещений
        self.W1 = np.random.randn(input_size, hidden1_size) * 0.1
        self.b1 = np.zeros((1, hidden1_size))

        self.W2 = np.random.randn(hidden1_size, hidden2_size) * 0.1
        self.b2 = np.zeros((1, hidden2_size))

        self.W3 = np.random.randn(hidden2_size, output_size) * 0.1
        self.b3 = np.zeros((1, output_size))

    def activation_func(self, x):
        return 1 / (1 + np.exp(-x))  # Сигмоида

    def activation_func_derivative(self, x):
        return x * (1 - x)  # Производная сигмоиды

    def forward(self, X):
        self.z1 = np.dot(X, self.W1) + self.b1
        self.a1 = self.activation_func(self.z1)

        self.z2 = np.dot(self.a1, self.W2) + self.b2
        self.a2 = self.activation_func(self.z2)

        self.z3 = np.dot(self.a2, self.W3) + self.b3
        self.a3 = self.activation_func(self.z3)

        return self.a3

    def backward(self, X, y):
        output_error = self.a3 - y  # Ошибка на выходе
        output_delta = output_error * self.activation_func_derivative(self.a3)

        hidden2_error = output_delta.dot(self.W3.T)
        hidden2_delta = hidden2_error * self.activation_func_derivative(self.a2)

        hidden1_error = hidden2_delta.dot(self.W2.T)
        hidden1_delta = hidden1_error * self.activation_func_derivative(self.a1)

        # Обновление весов и смещений
        self.W3 -= self.a2.T.dot(output_delta) * self.lr
        self.b3 -= np.sum(output_delta, axis=0, keepdims=True) * self.lr

        self.W2 -= self.a1.T.dot(hidden2_delta) * self.lr
        self.b2 -= np.sum(hidden2_delta, axis=0, keepdims=True) * self.lr

        self.W1 -= X.T.dot(hidden1_delta) * self.lr
        self.b1 -= np.sum(hidden1_delta, axis=0, keepdims=True) * self.lr

    def train(self, X, y, epochs):
        for epoch in range(epochs):
            self.forward(X)
            self.backward(X, y)
            if epoch % 100 == 0:
                loss = np.mean(np.square(y - self.a3))  # Среднеквадратичная ошибка
                print(f'Epoch {epoch}, Loss: {loss}')

Пример работы

In [10]:
# Подготовка данных
df2= pd.DataFrame(data)

X = df2[['feature1', 'feature2']].values
y = df2['label'].values.reshape(-1, 1)  # Преобразуем y в столбец

# Создаем и обучаем нейронную сеть
nn = NeuralNetwork(input_size=2, hidden1_size=3, hidden2_size=3, output_size=1, learning_rate=0.1)
nn.train(X, y, epochs=1000)

# Прогоняем данные через нейронную сеть
output = nn.forward(X)
print("Выход нейронной сети:", output)

Epoch 0, Loss: 0.25343579202470934
Epoch 100, Loss: 0.2400066653815962
Epoch 200, Loss: 0.24000605090893457
Epoch 300, Loss: 0.24000581026249956
Epoch 400, Loss: 0.24000557680809279
Epoch 500, Loss: 0.240005350185569
Epoch 600, Loss: 0.24000513006163615
Epoch 700, Loss: 0.24000491611605518
Epoch 800, Loss: 0.24000470804081556
Epoch 900, Loss: 0.24000450553935856
Выход нейронной сети: [[0.59996788]
 [0.59996429]
 [0.5999607 ]
 [0.59995711]
 [0.59995352]]


### Реализовать GPT как в п.2

In [73]:
class Head:
    def __init__(self, n_embd, head_size, learning_rate=0.01):
        self.n_embd = n_embd
        self.head_size = head_size
        self.lr = learning_rate

        # Инициализация весов для key, query, value
        self.key_weights = np.random.randn(n_embd, head_size) * 0.01
        self.query_weights = np.random.randn(n_embd, head_size) * 0.01
        self.value_weights = np.random.randn(n_embd, head_size) * 0.01
        # bias - False

        #создаем маску
    def create_mask(self, T):
        mask = np.tril(np.ones((T, T)), k=0)  # (T, T)
        return mask

    def forward(self, x):
        # x имеет размер (batch, time-step, channels)
        B, T, C = x.shape

        k = np.dot(x.reshape(-1, C), self.key_weights).reshape(B, T, -1)  # (B, T, head_size)
        q = np.dot(x.reshape(-1, C), self.query_weights).reshape(B, T, -1)  # (B, T, head_size)
        v = np.dot(x.reshape(-1, C), self.value_weights).reshape(B, T, -1)  # (B, T, head_size)

        # Вычисляем оценки внимания (affinities)
        wei = np.matmul(q, k.transpose(0, 2, 1)) * (k.shape[-1] ** -0.5  )  # (B, T, head_size) @ (B, head_size, T) -> (B, T, T)

        # Применяем маску
        mask = self.create_mask(T)  # (T, T)
        wei = np.where(mask == 0, -np.inf, wei)

        # Применяем softmax для получения весов внимания
        wei = np.exp(wei - np.max(wei, axis=-1, keepdims=True))
        wei /= np.sum(wei, axis=-1, keepdims=True)  # Нормализация

        # Выполняем взвешенное агрегирование value
        out = np.matmul(wei, v)  # (B, T, T) @ (B, T, head_size) -> (B, T, head_size)
        self.cache = (x, k, q, v, wei)

        return out


    def backward(self, d_out):
        x, k, q, v, wei = self.cache
        B, T, C = x.shape

        # Обратное распространение внимания и value
        d_wei = np.matmul(d_out, v.transpose(0, 2, 1))
        d_v = np.matmul(wei.transpose(0, 2, 1), d_out)

        d_wei = d_wei * (wei * (1 - wei)) # производная softmax

        # Обратное распространение key и query
        d_q = np.matmul(d_wei, k)
        d_k = np.matmul(d_wei.transpose(0, 2, 1), q)

        # Обновление весов
        self.key_weights -= np.dot(x.reshape(-1, C).T, d_k.reshape(-1, self.head_size)) * self.lr
        self.query_weights -= np.dot(x.reshape(-1, C).T, d_q.reshape(-1, self.head_size)) * self.lr
        self.value_weights -= np.dot(x.reshape(-1, C).T, d_v.reshape(-1, self.head_size)) * self.lr


        # Обучение
    def train(self, x, y, epochs):
        for epoch in range(epochs):
            out = self.forward(x)
            loss = np.mean((out - y)**2)
            d_out = 2 * (out - y) / y.size

            self.backward(d_out)

            if epoch % 100 == 0:
              print(f'Epoch {epoch}, Loss: {loss}')

Пример работы

In [74]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Пример текстовых данных
documents = [
    "Это первый документ.",
    "Это второй документ.",
    "И это третий документ.",
    "Это четвертый документ.",
]

# Создание TF-IDF векторов
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(documents).toarray()  # Преобразуем в массив NumPy
y = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1], [0, 0, 1]])  # Пример меток для обучения

# Параметры модели
n_embd = X.shape[1]  # Размерность векторов TF-IDF
head_size = 3  # Размер головы внимания
learning_rate = 0.01
epochs = 1000

# Создание экземпляра класса Head
head = Head(n_embd=n_embd, head_size=head_size, learning_rate=learning_rate)

# Изменяем форму X на (batch, time-step, channels)
X_reshaped = X[np.newaxis, :, :]
head.train(X_reshaped, y, epochs)

# Пример предсказания
test_document = ["Это новый документ для предсказания."]
X_test = vectorizer.transform(test_document).toarray()  # Преобразуем тестовый документ в TF-IDF вектор
X_test_reshaped = X_test[np.newaxis, :, :]
predicted_output = head.forward(X_test_reshaped)

print("Предсказанный выход:", predicted_output)

Epoch 0, Loss: 0.33566636582688875
Epoch 100, Loss: 0.25936675161929584
Epoch 200, Loss: 0.21936288722117495
Epoch 300, Loss: 0.1960606110326333
Epoch 400, Loss: 0.18063681682768895
Epoch 500, Loss: 0.16910243259167237
Epoch 600, Loss: 0.15963926116590516
Epoch 700, Loss: 0.15140525428689153
Epoch 800, Loss: 0.1439984369547039
Epoch 900, Loss: 0.13721627160847516
Предсказанный выход: [[[0.15876145 0.23338705 0.62459387]]]
